In [1]:
from typing import Iterator, Optional
import re

In [2]:
class Tok:
    def __init__(self, name: str, value: str) -> None:
        self.name = name
        self.value = value

In [3]:
class Tokenizer:
    TOKPATTERN = re.compile("\s*(?:(\d+)|(.))")

    def __init__(self, source: str) -> None:
        self._tokgen = self._gen_tokens(source)
        self.cur_token: Tok | None = None

    def get_next_token(self) -> Optional[Tok]:
        try:
            self.cur_token = next(self._tokgen)
        except StopIteration:
            self.cur_token = None
        return self.cur_token

    def _gen_tokens(self, source: str) -> Iterator[Tok]:
        number: str
        operator: str
        for number, operator in self.TOKPATTERN.findall(source):
            if number:
                yield Tok('NUMBER', number)
            elif operator == '(':
                yield Tok('LEFTPAREN', '(')
            elif operator == ')':
                yield Tok('RIGHTPAREN', ')')
            else:
                yield Tok('BINOP', operator)

In [4]:
class OpInfo:
    def __init__(self, prec, assoc):
        self.prec = prec
        self.assoc = assoc

In [5]:
OPINFO_MAP = {
    '+':    OpInfo(1, 'LEFT'),
    '-':    OpInfo(1, 'LEFT'),
    '*':    OpInfo(2, 'LEFT'),
    '/':    OpInfo(2, 'LEFT'),
    '^':    OpInfo(3, 'RIGHT'),
}

In [6]:
def parse_error(msg):
    print(msg)
    exit(code=1)

In [7]:
def compute_expr():
    pass

In [8]:
def compute_atom(tokenizer):
    tok = tokenizer.cur_token
    if tok.name == 'LEFTPAREN':
        tokenizer.get_next_token()
        val = compute_expr(tokenizer, 1)
        if tokenizer.cur_token.name != 'RIGHTPAREN':
            parse_error('unmatched "("')
        tokenizer.get_next_token()
        return val
    elif tok is None:
            parse_error('source ended unexpectedly')
    elif tok.name == 'BINOP':
        parse_error('expected an atom, not an operator "%s"' % tok.value)
    else:
        assert tok.name == 'NUMBER'
        tokenizer.get_next_token()
        return int(tok.value)

In [9]:
def compute_op(op, lhs, rhs):
    lhs = int(lhs); rhs = int(rhs)
    if op == '+':   return lhs + rhs
    elif op == '-': return lhs - rhs
    elif op == '*': return lhs * rhs
    elif op == '/': return lhs / rhs
    elif op == '^': return lhs ** rhs
    else:
        parse_error('unknown operator "%s"' % op)

In [10]:
def compute_expr(tokenizer, min_prec):
    atom_lhs = compute_atom(tokenizer)

    while True:
        cur = tokenizer.cur_token
        if (cur is None or cur.name != 'BINOP'
                        or OPINFO_MAP[cur.value].prec < min_prec):
            break

        # Inside this loop the current token is a binary operator
        assert cur.name == 'BINOP'

        # Get the operator's precedence and associativity, and compute a
        # minimal precedence for the recursive call
        op = cur.value
        prec = OPINFO_MAP[op].prec
        assoc = OPINFO_MAP[op].assoc
        next_min_prec = prec + 1 if assoc == 'LEFT' else prec

        # Consume the current token and prepare the next one for the
        # recursive call
        tokenizer.get_next_token()
        atom_rhs = compute_expr(tokenizer, next_min_prec)

        # Update lhs with the new value
        atom_lhs = compute_op(op, atom_lhs, atom_rhs)

    return atom_lhs

In [14]:
tokenizer = Tokenizer('1+2*3+4')
tokenizer.get_next_token()
compute_expr(tokenizer, 1)

11